In [5]:
import os
project_name = "reco-chef"; branch = "30music"; account = "sparsh-ai"
project_path = os.path.join('/content', project_name)

In [6]:
if not os.path.exists(project_path):
    !pip install -U -q dvc dvc[gdrive]
    !cp -r /content/drive/MyDrive/git_credentials/. ~
    path = "/content/" + project_name; 
    !mkdir "{path}"
    %cd "{path}"
    !git init
    !git remote add origin https://github.com/"{account}"/"{project_name}".git
    !git pull origin "{branch}"
    !git checkout "{branch}"
else:
    %cd "{project_path}"

/content/reco-chef
Initialized empty Git repository in /content/reco-chef/.git/
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 188 (delta 58), reused 170 (delta 41), pack-reused 0
Receiving objects: 100% (188/188), 49.19 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (58/58), done.
From https://github.com/sparsh-ai/reco-chef
 * branch            30music    -> FETCH_HEAD
 * [new branch]      30music    -> origin/30music
Branch '30music' set up to track remote branch '30music' from 'origin'.
Switched to a new branch '30music'


In [39]:
!git status

On branch 30music
Your branch is up to date with 'origin/30music'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   dvc.lock
	modified:   dvc.yaml
	modified:   params.yaml
	deleted:    src/dataprep/30music.py
	deleted:    src/dataprep/__init__.py
	modified:   src/models/__init__.py
	deleted:    src/train/__init__.py
	deleted:    src/train/itempop.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	src/prepare.py
	src/train.py

no changes added to commit (use "git add" and/or "git commit -a")


In [40]:
!git add . && git commit -m 'commit' && git push origin "{branch}"

usic f57d404] commit
 9 files changed, 81 insertions(+), 36 deletions(-)
 rewrite dvc.yaml (97%)
 delete mode 100644 src/dataprep/__init__.py
 rename src/{dataprep/30music.py => prepare.py} (85%)
 create mode 100644 src/train.py
 delete mode 100644 src/train/__init__.py
 delete mode 100644 src/train/itempop.py
Counting objects: 10, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (10/10), 2.06 KiB | 2.06 MiB/s, done.
Total 10 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/sparsh-ai/reco-chef.git
   d6ec8d0..f57d404  30music -> 30music


In [9]:
!make setup

pip install -e .
Obtaining file:///content/reco-chef
  Running setup.py develop for recotut


In [10]:
%reload_ext autoreload
%autoreload 2

In [ ]:
!dvc pull ./data/bronze/30music/sessions_sample_10.parquet.snappy.dvc

In [ ]:
!dvc repro

In [37]:
%%writefile ./src/prepare.py

import os
import sys
import yaml
import pandas as pd

from src.dataset import SessionDataset

params = yaml.safe_load(open("params.yaml"))["prepare"]

if len(sys.argv) != 3:
    sys.stderr.write("Arguments error. Usage:\n")
    sys.stderr.write("\tpython {dataprep}.py {data-source-filepath} {data-target-dirpath}\n")
    sys.exit(1)

filter_last_months = params['filter_last_months']
filter_topk = params['filter_topk']

data_source_path = str(sys.argv[1])
data_target_path = str(sys.argv[2])
data_target_path_train = os.path.join(data_target_path, 'train.parquet.snappy')
data_target_path_test = os.path.join(data_target_path, 'test.parquet.snappy')


def prepare_data(data_source_path,
                 filter_last_months, filter_topk,
                 data_target_path_train, data_target_path_test):
    
    df = pd.read_parquet(data_source_path)
    sess_ds = SessionDataset(df)

    sess_ds.filter_by_time(filter_last_months)
    sess_ds.convert_to_sequence(filter_topk)
    stats = sess_ds.get_stats()
    print(stats)
    sess_ds.last_session_out_split()
    print("Train sessions: {} - Test sessions: {}".format(len(sess_ds.train), len(sess_ds.test)))
    sess_ds.train.to_parquet(data_target_path_train, compression='snappy')
    sess_ds.test.to_parquet(data_target_path_test, compression='snappy')


os.makedirs(data_target_path, exist_ok=True)

prepare_data(data_source_path,
             filter_last_months, filter_topk,
             data_target_path_train, data_target_path_test)

Overwriting ./src/prepare.py


In [36]:
%%writefile ./src/train.py
import os
import sys
import yaml
import pickle
import pandas as pd

from src.models import PopularityRecommender

params = yaml.safe_load(open("params.yaml"))["train"]

models = {
    'itempop': PopularityRecommender()
}


def load_data(path):
    path_train = os.path.join(path, 'train.parquet.snappy')
    path_test = os.path.join(path, 'test.parquet.snappy')
    train = pd.read_parquet(path_train)
    test = pd.read_parquet(path_test)
    return train, test


def load_model(name):
    model = models[name]
    return model


if __name__ == "__main__":
    model_name = params['model_name']
    data_path = str(sys.argv[1])
    model_path = str(sys.argv[2])
    train, test = load_data(data_path)
    model = load_model(model_name)
    model.fit(train)
    pickle.dump(model, open(os.path.join(model_path, model_name+'.pkl'), 'wb'))

Overwriting ./src/train.py


## Scripts

In [28]:
!python ./src/train.py ./data/silver/30music ./artifacts/30music/models

In [ ]:
!python ./src/eval/itempop.py \
./data/silver/30music \
./artifacts/30music/models/itempop.pkl

In [ ]:
import pickle
results = pickle.load(open('./artifacts/30music/results/itempop.pkl', 'rb'))
results

## DVC Pipeline

In [24]:
%%writefile params.yaml

prepare:
  filter_last_months: 1
  filter_topk: 1000

train:
  model_name: itempop

Overwriting params.yaml


In [38]:
%%writefile dvc.yaml
stages:
    prepare:
        cmd: python src/prepare.py data/bronze/30music/sessions_sample_10.parquet.snappy data/silver/30music
        deps:
        - data/bronze/30music/sessions_sample_10.parquet.snappy
        - src/dataprep/30music.py
        params:
        - prepare.filter_last_months
        - prepare.filter_topk
        outs:
        - data/silver/30music
    train:
        cmd: python src/train.py data/silver/30music artifacts/30music/models
        deps:
        - data/silver/30music/test.parquet.snappy
        - data/silver/30music/train.parquet.snappy
        - src/train.py
        params:
        - train.model_name
    evaluate:
        cmd: python

Overwriting dvc.yaml
